In [20]:
from datetime import datetime, timedelta
mindate = datetime.strptime('2021-01-01','%Y-%m-%d')
maxdate = datetime.strptime('2021-12-31','%Y-%m-%d')
daterange = [ (mindate + timedelta(hours=x)).strftime('%Y%m%d %H') for x in range(0,(maxdate-mindate).days)]

print(daterange[:10])



['20210101 00', '20210101 01', '20210101 02', '20210101 03', '20210101 04', '20210101 05', '20210101 06', '20210101 07', '20210101 08', '20210101 09']


In [10]:
import pandas as pd
dt_index = pd.date_range(start='20210101', end='20211231')
dt_list = dt_index.strftime('%Y-%m-%d').tolist()
print(dt_list[:10])

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10']


In [9]:
# tier1 데이터 sql
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)
data_path = '/home/data/data-warehouse/tier1/event/2020-12-01/00'
attr = [
    '*',
]
df = spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path) \
    .selectExpr(attr)
df.show(3)

TypeError: unexpected item type: <class 'slice'>

In [1]:
# raw 데이터 sql
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)
data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-branch/2020-12/01/00/*'
# data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-adbrix/2020-12/01/00/*'
# data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-singular/2021-01/01/00/*'
pre_branch_attr = [
    '"BRANCH"                                        AS tracking',
    'trackingObject.organization_name[0]             AS trackingId',
    'CASE WHEN trackingObject.name[0] = "SEARCH"   THEN "TPD007" \
            WHEN trackingObject.name[0] = "VIEW_ITEM"  THEN "TPD002" \
            WHEN trackingObject.name[0] = "ADD_TO_CART" THEN "TPD003" \
            WHEN trackingObject.name[0] = "PURCHASE" THEN "TPD004"\
            WHEN trackingObject.name[0] = "INSTALL"    THEN "TPD008"\
            WHEN trackingObject.name[0] = "OPEN" THEN "TPD009"  ELSE "TPD999" END     AS trackingEventCode',
    'CASE WHEN trackingObject.ud_os[0] = "ANDROID" THEN trackingObject.ud_aaid[0] \
            WHEN trackingObject.ud_os[0] = "IOS" THEN trackingObject.ud_idfa[0] END                     AS cid',
    'CASE WHEN trackingObject.ud_os[0] = "ANDROID" THEN "ATC001" \
            WHEN trackingObject.ud_os[0] = "IOS" THEN "ATC002" ELSE "ATC999" END      AS osTypeCode',
    'logTimeStamp                  ',
    'trackingObject.event_timestamp[0]               AS eventTimeStamp',
    'trackingObject.latd_campaign[0]                 AS campaign',
    'trackingObject.ci0_product_name[0]              AS contentId',
    '""                                              AS contentName',
    'trackingObject.ci0_price[0]                     AS value',
    'trackingObject.ci0_quantity[0]                  AS quantity',
    'trackingObject.ci0_price[0] * trackingObject.ci0_quantity[0]     AS amount',
    'trackingObject.ci0_currency[0]                  AS currency',
    '""                                              AS activityParam',
    'trackingObject.attributed[0]                    AS attributed',
    'trackingObject.latd_advertising_partner_name[0] AS latdAdvertisingPartnerName',
]
pre_adbrix_attr = [  
    '"ADBRIX"                                      AS tracking',
    'trackingObject.package_name[0]                AS trackingId',      
    'CASE WHEN trackingObject.activity[0] = "search"  THEN "TPD001" \
            WHEN trackingObject.activity[0] = "product_view"  THEN "TPD002" \
            WHEN trackingObject.activity[0] = "add_to_cart" THEN "TPD003" \
            WHEN trackingObject.activity[0] = "purchase" THEN "TPD004" \
            WHEN trackingObject.activity[0] = "sign_up" THEN "TPD012" \
            WHEN trackingObject.activity[0] = "refund" THEN "TPD015" \
            WHEN trackingObject.activity[0] = "add_to_wishlist" THEN "TPD014" \
            WHEN trackingObject.activity[0] = "login" THEN "TPD013" ELSE "TPD999" END  AS trackingEventCode',
    'CASE WHEN trackingObject.platform[0] = "android" THEN trackingObject.gaid[0] ELSE trackingObject.ifa[0] END AS cid',
    'CASE WHEN trackingObject.platform[0] = "android" THEN "ATC001" \
            WHEN trackingObject.platform[0] = "ios" THEN "ATC002" ELSE "ATC999" END AS osTypeCode', 
    'CAST(logTimeStamp AS timestamp) AS logTimeStamp',
    'CAST(trackingObject.event_time[0] * 1000 AS timestamp)    AS eventTimeStamp',
    '""                                            as campaign',
    'trackingObject.product_id[0]                  AS contentId',
    'trackingObject.product_name[0]                AS contentName',
    'cast(trackingObject.price[0] as integer)                       AS value',
    'cast(trackingObject.quantity[0] as integer)                    AS quantity',
    'cast(trackingObject.sales[0] as integer)                       AS amount',
    'trackingObject.currency[0]                    AS currency',
    'trackingObject.activity_param[0]              AS activityParam',
    '""                                            AS attributed',
    '""                                            AS latdAdvertisingPartnerName',      
]
pre_singular_attr = [
    '"SINGULAR"                                               AS tracking',
    'trackingObject.app_name[0]                               AS trackingid',
    'CASE WHEN trackingObject.evtname[0] = "ViewItem"  THEN "TPD002"\
            WHEN trackingObject.evtname[0] = "AddToCart" THEN "TPD003"\
            WHEN trackingObject.evtname[0] = "Purchase"  THEN "TPD004"\
            WHEN trackingObject.evtname[0] = "FirstPurchase"  THEN "TPD010"\
            WHEN trackingObject.evtname[0] = "__START__" THEN "TPD008"\
            WHEN trackingObject.evtname[0] = "__SESSION__"  THEN "TPD009"\
            WHEN trackingObject.evtname[0] = "__iap__"   THEN "TPD011"   ELSE "TPD999" END AS trackingEventCode',
    'CASE WHEN trackingObject.platform[0] = "Android" THEN trackingObject.aifa[0] ELSE trackingObject.idfa[0] END AS cid',
    'CASE WHEN trackingObject.platform[0] = "Android" THEN "ATC001"\
            WHEN trackingObject.platform[0] = "iOS" THEN "ATC002"  ELSE "ATC999" END  AS osTypeCode',
    'logTimeStamp',
    'CAST(trackingObject.utc[0] * 1000 AS String)             AS eventTimeStamp',
    'trackingObject.cid[0]                                    AS campaign',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].id") ELSE GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].prd_code") END AS contentId',
    'GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].product")  AS contentName',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].price") ELSE "0" END  AS value',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].quantity") ELSE "0" END AS quantity',
    'trackingObject.amount[0]                                 as amount',
    'trackingObject.currency[0]                               as currency',
    'trackingObject.evtattr[0]                                AS activityParam',
    'trackingObject.is_re_eng[0]                              AS attributed',
    '""                                                       AS latdAdvertisingPartnerName',
]

# 스키마 지정해서 불러들여야하는데... 에러남
union_schema = StructType([
    StructField('tracking', StringType(), True),
    StructField('trackingId', StringType(), True),
    StructField('trackingEventCode', StringType(), True),
    StructField('cid', StringType(), True),
    StructField('osTypeCode', StringType(), True),
    StructField('logTimeStamp', TimestampType(), True),
    StructField('eventTimeStamp', TimestampType(), True),
    StructField('campaign', StringType(), True),
    StructField('contentId', StringType(), True),
    StructField('contentName', StringType(), True),
    StructField('value', IntegerType(), True),
    StructField('quantity', IntegerType(), True),
    StructField('amount', IntegerType(), True),
    StructField('currency', StringType(), True),
    StructField('activityParam', StringType(), True),
    StructField('attributed', StringType(), True),
    StructField('latdAdvertisingPartnerName', StringType(), True),
])


df = spark.read\
    .format('json')\
    .option('header',True)\
    .load(data_path)\
    .selectExpr(pre_adbrix_attr)\

root
 |-- tracking: string (nullable = false)
 |-- trackingId: string (nullable = true)
 |-- trackingEventCode: string (nullable = false)
 |-- cid: string (nullable = true)
 |-- osTypeCode: string (nullable = false)
 |-- logTimeStamp: timestamp (nullable = true)
 |-- eventTimeStamp: timestamp (nullable = true)
 |-- campaign: string (nullable = false)
 |-- contentId: string (nullable = true)
 |-- contentName: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- activityParam: string (nullable = true)
 |-- attributed: string (nullable = false)
 |-- latdAdvertisingPartnerName: string (nullable = false)



In [6]:
df.printSchema()

root
 |-- tracking: string (nullable = false)
 |-- trackingId: string (nullable = true)
 |-- trackingEventCode: string (nullable = false)
 |-- cid: string (nullable = true)
 |-- osTypeCode: string (nullable = false)
 |-- logTimeStamp: timestamp (nullable = true)
 |-- eventTimeStamp: timestamp (nullable = true)
 |-- campaign: string (nullable = false)
 |-- contentId: string (nullable = true)
 |-- contentName: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- activityParam: string (nullable = true)
 |-- attributed: string (nullable = false)
 |-- latdAdvertisingPartnerName: string (nullable = false)



In [ ]:
df.show(3,False)

In [5]:
# write
write_path = '/home/data/temp/adbrix'
df.where('trackingId is not NULL')\
    .sort('eventTimeStamp')\
    .coalesce(20)\
    .write\
    .option('compression','gzip')\
    .option('parquet.enable.dictionary', 'true')\
    .option('parquet.block.size', f'{128*1024*1024}')\
    .option('parquet.page.size', f'{2*1024*1024}')\
    .option('parquet.dictionary.page.size', f'{8*1024*1024}')\
    .mode('overwrite')\
    .parquet(write_path)

Py4JJavaError: An error occurred while calling o95.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at org

In [ ]:
# read and print schema
df2 = spark.read\
    .format('parquet')\
    .option('header',True)\
    .load(write_path)    

df2.printSchema()

In [ ]:
import os
os.listdir('/home/data/data-warehouse/tier1/event/')